In [8]:
import pandas as pd
import json

data={
    'cmd':'Buy',
    'stock':'INFY',
    'qty':'12',
    'type':'BO',
    'SL':'1234',
    'Target':'1234',
    'Price':'1234',
    'state':'idle'
}

df = pd.DataFrame(data=data, index=['stock'])
json_data = df.to_json(orient='records')

In [9]:
json_data

'[{"cmd":"Buy","stock":"INFY","qty":"12","type":"BO","SL":"1234","Target":"1234","Price":"1234","state":"idle"}]'

In [ ]:
from kiteconnect import KiteConnect
from kiteconnect import KiteTicker

In [ ]:
kite = KiteConnect(api_key=KiteAPIKey)
reauthentication = False

f = open("kite_data/access_token.txt", mode="r")
access_token = f.readlines()
logger.info(access_token[0])

try:
    kite.set_access_token(access_token[0])
    logger.info("Welcome "+kite.profile()['user_name'])
except:
    logger.critical("Offline Mode: Could not authenticate with the Kite Server")
    offline = True

In [ ]:
try:
    if exchange=="":
        exchange = "NSE"
except:
    logging.debug("Exchange not defined: Using default NSE")
    exchange = "NSE"

try:
    instruments_df = getInstruments(exchange)
    instruments_df.to_hdf('kite_data/kite_cache.h5', key=exchange, mode='a', format="table")
except:
    logger.critical("Error in downloading instrument table from kite")
    
try:
    instruments_df = pd.read_hdf('kite_data/kite_cache.h5', key=exchange, mode='r', format="table")

    EQSYMBOL = lambda x:instruments_df[instruments_df['instrument_token']==x].index[0]
    EQTOKEN = lambda x:instruments_df.loc[x,'instrument_token']
except:
    logger.critical("Error in reading h5 file")

In [ ]:
#logging.critical("BUY"+symbol)
def buy_slm(symbol, price, trigger,quantity=1): 
    logger.info('%12s'%"BUY SLM: "+symbol+", price: "+str('%0.2f'%price)+", stoploss: "+str('%0.2f'%stoploss)+", quantity: "+str(quantity))
    
    if papertrade:
        return
    
    try:
        order_id = kite.place_order(tradingsymbol=symbol,
                                exchange=kite.EXCHANGE_NSE,
                                transaction_type=kite.TRANSACTION_TYPE_BUY,
                                quantity=quantity,
                                order_type=kite.ORDER_TYPE_SLM,
                                product=kite.PRODUCT_MIS,
                                trigger_price=round(trigger,1),
                                #stoploss=round(stoploss,1),
                                #price=price,
                                variety=kite.VARIETY_REGULAR
                                )
        logger.info("Order placed. ID is: {}".format(order_id))
    except Exception as e:
        logger.info("Order placement failed: {}".format(e.message))
        
def sell_slm(symbol, price, trigger, quantity=1):
    
    logger.info('%12s'%"SELL SLM: "+symbol+", price: "+str('%0.2f'%price)+", stoploss: "+str('%0.2f'%stoploss)+", quantity: "+str(quantity))
       
    if papertrade:
         return
    try:
        order_id = kite.place_order(tradingsymbol=symbol,
                            exchange=kite.EXCHANGE_NSE,
                            transaction_type=kite.TRANSACTION_TYPE_SELL,
                            quantity=quantity,
                            order_type=kite.ORDER_TYPE_SLM,
                            product=kite.PRODUCT_MIS,
                            trigger_price=round(trigger,1),
                            #price=price,
                            variety=kite.VARIETY_REGULAR)
        logger.info("Order placed. ID is: {}".format(order_id))
    except Exception as e:
        logger.info("Order placement failed: {}".format(e.message))

def buy_bo(symbol, price, trigger, stoploss, squareoff, quantity=1, tag="bot"): 
    logger.info('%12s'%"BUY BO: "+symbol+", price: "+str('%0.2f'%price)+", squareoff: "+str('%0.2f'%squareoff)+", stoploss: "+str('%0.2f'%stoploss)+", quantity: "+str(quantity))
    if papertrade:
        return
    
    try:
        order_id = kite.place_order(tradingsymbol=symbol, exchange=kite.EXCHANGE_NSE, transaction_type=kite.TRANSACTION_TYPE_BUY,
                        order_type=kite.ORDER_TYPE_LIMIT, product=kite.PRODUCT_MIS, variety=kite.VARIETY_BO, 
                                quantity=quantity, trigger_price=trigger, price=price,
                                squareoff=squareoff,  stoploss=stoploss, tag=tag )
        logger.info("Order placed. ID is: {}".format(order_id))
    except Exception as e:
        logger.info("Order placement failed: {}".format(e.message))



def sell_bo(symbol, price, trigger, stoploss, squareoff, quantity=1, tag="bot"): 
    logger.info('%12s'%"SELL BO: "+symbol+", price: "+str('%0.2f'%price)+", squareoff: "+str('%0.2f'%squareoff)+", stoploss: "+str('%0.2f'%stoploss)+", quantity: "+str(quantity))
    if papertrade:
        return
    
    try:
        order_id = kite.place_order(tradingsymbol=symbol, exchange=kite.EXCHANGE_NSE, transaction_type=kite.TRANSACTION_TYPE_SELL,
                                order_type=kite.ORDER_TYPE_LIMIT, product=kite.PRODUCT_MIS, variety=kite.VARIETY_BO,
                                quantity=quantity, trigger_price=trigger, price=price,
                                stoploss=stoploss, squareoff=squareoff,  tag=tag )
        logger.info("Order placed. ID is: {}".format(order_id))
    except Exception as e:
        logger.info("Order placement failed: {}".format(e.message))
        
def getOrders():    
    # Fetch all orders
    return pd.DataFrame(kite.orders())

def cancelOrder(orderId):
    if papertrade:
        logging.critical("In Paper Trade Mode: Order cancellation not possible")
        return
    
    try:
        kite.cancel_order(variety=kite.VARIETY_REGULAR, order_id=orderId, parent_order_id=None)    
    except Exception as e:
        logger.info("Order Cancellation failed: {}".format(e.message))
        
def squareoff(symbol=None, tag="bot"):
    logger.info('%12s'%"Squareoff: "+symbol)
    if papertrade:
        return
    
    orders_df = pd.DataFrame(kite.orders())
    if symbol != None:
        open_orders = orders_df[(orders_df['tradingsymbol']==symbol) & (orders_df['status'] == 'TRIGGER PENDING')  & (orders_df['tag'] == tag)]
    else:
        open_orders = orders_df[(orders_df['status'] == 'TRIGGER PENDING')  & (orders_df['tag'] == tag)]
        
    for index, row in open_orders.iterrows():
        print(row.order_id, row.parent_order_id)
        #kite.exit_order(variety=kite.VARIETY_AMO, order_id=row.order_id, parent_order_id=row.parent_order_id)
        kite.exit_order(variety=kite.VARIETY_BO, order_id=order_id, parent_order_id=parent_order_id)


In [ ]:
def resample(ws, freq="1min"):
    #F = open("kite_data/recommendation.csv","a") 
    
    logging.debug(str(ws.prevtimeStamp)+": In resampler function")
    
    if ws.LiveStream.empty:
        logging.debug(str(ws.prevtimeStamp)+": Empty dataframe, Exiting resampler")
        return
      
    LiveStream2 = ws.LiveStream.groupby(['symbol','date']).agg({'price':['first','max','min','last'], 'volume':['last']})
    LiveStream2.columns = LiveStream2.columns.droplevel()
    LiveStream2.columns = ['open', 'high','low','close', 'volume']

    for index, data in LiveStream2.groupby(level=0):
        sampled = data.loc[index].resample(freq).agg({'open':{'open':'first'},'high':{'high':'max'},'low':{'low':'min'},'close':{'close':'last'},'volume':{'volume':'last'}})
        sampled.columns = sampled.columns.droplevel()
        logger.debug(index)
        
        sampled['volume'] = sampled['volume'] - sampled['volume'].shift(1) 
        sampled['symbol'] = index
        sampled.set_index(['symbol',sampled.index], inplace=True)
        #logger.debug(sampled.tail())

        ws.LiveStreamOHLC = ws.LiveStreamOHLC.append(sampled.iloc[-1])
        
    #ws.LiveStreamOHLC.to_csv("kite_data/livestreamohlc.csv", mode='a')

    for symbol in portfolio[0]:
        #symbol = portfolio[0].iloc[-1]
        temp_ohlc_df = ws.LiveStreamOHLC.loc[symbol].tail(120)
        ws.tradebook_df.loc[symbol,'symbol'].trade_manager(symbol, temp_ohlc_df)
    
    
def ticksHandler(ws, ticks):
    #timeStamp = dt.datetime.now().replace(second=0, microsecond=0)
    tick_df = pd.DataFrame(ticks)
    
    try:
        #tick_df.loc[tick_df['timestamp'].isna(), 'timestamp'] = timeStamp
        tick_df = tick_df[['timestamp','instrument_token','last_price','volume']]
        tick_df.instrument_token = tick_df.instrument_token.apply(EQSYMBOL)
        tick_df.columns = ['date','symbol','price','volume']
        tick_df.set_index(['symbol','date'], inplace=True)
        
        timeStamp = tick_df.index[0][-1].to_pydatetime()
        
    except  Exception as e:
        logging.debug("Exception: ticksHandler: "+str(e)+str(tick_df))
        
    if( (timeStamp - ws.prevtimeStamp) >= dt.timedelta(minutes=1)):
        ws.prevtimeStamp = timeStamp
        resample(ws)
    
    ws.LiveStream = ws.LiveStream.append(tick_df)

def orderNotification(ws,data):
    #logger.debug(data)
    order_df = pd.DataFrame.from_dict(data, orient='index')

    symbol = order_df.loc['tradingsymbol'][0]
    
    ws.tradebook_df.loc[symbol,'symbol'].update_order(order_df)
    #logger.debug(order_df)

def initTrade(ws):
    ws.prevtimeStamp = dt.datetime.now() - dt.timedelta(minutes=10)
    toDate = dt.datetime.now()
    
    ws.tradebook_df = pd.DataFrame()
    
    for symbol in portfolio[0]:
        temp_df = pd.DataFrame(data=[algoTrade(symbol)], index=[symbol], columns=['symbol'])
        ws.tradebook_df = ws.tradebook_df.append(temp_df)
        
    #TODO: Convert to multistock handling
    #symbol = portfolio[0].iloc[-1]
    #ws.a = algoTrade(symbol)
    
    ws.LiveStream = pd.DataFrame()
    ws.LiveStreamOHLC = pd.DataFrame()
    ws.LiveStreamOHLC = portfolioDownload(portfolio, toDate)
    

In [ ]:
def on_ticks(ws, ticks):
    # Callback to receive ticks.
    #logging.debug("Ticks: {}".format(ticks))
    ticksHandler(ws, ticks)


def on_connect(ws, response):
    initTrade(ws)
    logger.debug(portfolioToken)
    # Callback on successful connect.
    # Subscribe to a list of instrument_tokens (RELIANCE and ACC here).
    #ws.subscribe(portfolioToken)

    ws.subscribe(portfolioToken)
    
    # Set RELIANCE to tick in `full` mode.
    # MODE_LTP, MODE_QUOTE, or MODE_FULL

    ws.set_mode(ws.MODE_FULL, portfolioToken)
    #ws.set_mode(ws.MODE_FULL, [225537]) 
    #ws.set_mode(ws.MODE_LTP, [225537, 3861249]) 
    #ws.set_mode(ws.MODE_MODE_QUOTE, [2714625,779521]) 

def on_close(ws, code, reason):
    # On connection close stop the main loop
    # Reconnection will not happen after executing `ws.stop()`
    ws.stop()

def on_order_update(ws, data):
    #logger.info("New Order Update")
    orderNotification(ws,data)